In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms

In [2]:
# 加载数据集
batch_size = 200

learning_rate = 0.01

epochs = 10

train_loader = torch.utils.data.DataLoader(
datasets.MNIST('Mnist_data',train = True,download = True,
                transform = transforms.Compose([
                    #  将数据变换为张量形式
                    transforms.ToTensor(),
                    # 根据官方提供的均值和标准差,标准化数据方便减少梯度下降次数
                    transforms.Normalize((0.1307,), (0.3081,))                  
                ])),batch_size = batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(
datasets.MNIST('Mnist_data',train = False,download = True,
              transform = transforms.Compose([
                  transforms.ToTensor(),
                  transforms.Normalize((0.1307,), (0.3081,))
              ])),batch_size = batch_size,shuffle = True)

100.1%

Extracting Mnist_data\MNIST\raw\train-images-idx3-ubyte.gz


113.5%

Extracting Mnist_data\MNIST\raw\train-labels-idx1-ubyte.gz


100.4%

Extracting Mnist_data\MNIST\raw\t10k-images-idx3-ubyte.gz


180.4%

Extracting Mnist_data\MNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [38]:
print(type(train_loader))
print(train_loader.dataset)
#查看数据结构
for i in enumerate(train_loader):
    print(i)
    break

<class 'torch.utils.data.dataloader.DataLoader'>
Dataset MNIST
    Number of datapoints: 60000
    Root location: Mnist_data
    Split: Train
(0, [tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242

In [39]:
class First_nn(nn.Module):
    def __init__(self):
        super(First_nn,self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(28*28,200),
            #  因为数据被标准化了可能为负值为防止梯度为零用LeakyReLU
            nn.LeakyReLU(inplace = True),
            nn.Linear(200,200),
            nn.LeakyReLU(inplace = True),
            nn.Linear(200,10),
            nn.LeakyReLU(inplace = True)
            )
    def forward(self,x):
        return self.model(x)
# 将模型搬到GPU上
device = torch.device('cuda:0')
#net = First_nn().cuda()
net = First_nn().to(device)
#随机梯度下降
#Error:optimizer = optim.SGD(net.parameters(),lr = learning_rate).to(device)
optimizer = optim.SGD(net.parameters(),lr = learning_rate)
# softmax激活 + CrossEntropy损失
loss_func = nn.CrossEntropyLoss().to(device)

for epoch in range(epochs):
    for batch_idx,(data,target) in enumerate(train_loader):
        data = data.view(data.size(0),1*28*28)
        data = data.to(device)
        target = target.cuda()
        logits = net(data)
        loss = loss_func(logits,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (batch_idx+1)%100 ==0:
            print('Train epoch {} [{}/{}({:.2f}%)]\t loss:{}'.format(
                    epoch,(batch_idx+1)*len(data),len(train_loader.dataset),
                    100.*(batch_idx+1)*len(data)/len(train_loader.dataset),loss.item()))
        
        if batch_idx == 0:
            print('Train epoch {} [{}/{}({}%)]\t loss:{}'.format(
                   epoch,batch_idx*len(data),len(train_loader.dataset),
                   batch_idx/len(train_loader.dataset),loss.item()))
    # test module
    test_loss = 0
    correct = 0
    for batch_idx,(data,target) in enumerate(test_loader):
        data = data.view(data.size(0),28*28)
        data = data.to(device)
        target = target.cuda()
        logits = net(data)
        test_loss += loss_func(logits,target).item()
        predict = logits.argmax(dim=1)
        correct += predict.eq(target).float().sum().item() 
    test_loss /= len(test_loader.dataset)
    print('epoch:{}  test_data Average_loss : {}\t Accuracy: {}/{}({:2f})%'.format(
              epoch,test_loss,correct,len(test_loader.dataset),100.*correct/len(test_loader.dataset)))


Train epoch 0 [0/60000(0.0%)]	 loss:2.306537389755249
Train epoch 0 [20000/60000(33.33%)]	 loss:2.035033702850342
Train epoch 0 [40000/60000(66.67%)]	 loss:1.2957611083984375
Train epoch 0 [60000/60000(100.00%)]	 loss:0.7836695909500122
epoch:0  test_data Average_loss : 0.003627978050708771	 Accuracy: 8302.0/10000(83.020000)%
Train epoch 1 [0/60000(0.0%)]	 loss:0.7361741065979004
Train epoch 1 [20000/60000(33.33%)]	 loss:0.49886584281921387
Train epoch 1 [40000/60000(66.67%)]	 loss:0.49449071288108826
Train epoch 1 [60000/60000(100.00%)]	 loss:0.3523230254650116
epoch:1  test_data Average_loss : 0.002026969975233078	 Accuracy: 8896.0/10000(88.960000)%
Train epoch 2 [0/60000(0.0%)]	 loss:0.4014418125152588
Train epoch 2 [20000/60000(33.33%)]	 loss:0.3810940682888031
Train epoch 2 [40000/60000(66.67%)]	 loss:0.44071638584136963
Train epoch 2 [60000/60000(100.00%)]	 loss:0.3752475082874298
epoch:2  test_data Average_loss : 0.0016781002402305603	 Accuracy: 9025.0/10000(90.250000)%
Train ep

In [1]:
print(a)

NameError: name 'a' is not defined